In [1]:
import pandas as pd

In [134]:
df = pd.read_html('data/examples/20201219_Auszug.html')
df = df[0]
df.columns = df.loc[0]
df = df[1:]
df = df.reset_index(drop=True)

In [164]:
df['Menge'] = df['Text'].str.extract('((?<=MENGE )[\d,]+)')
df['Menge'] = df['Menge'].str.replace('.', '').str.replace(',', '.')

df['Kurs'] = df['Text'].str.extract('((?<=KURS )[\d,]+)')
df['Kurs'] = df['Kurs'].str.replace('.', '').str.replace(',', '.')

df['DEVISENKURS'] = df['Text'].str.extract('((?<=DEVISENKURS )[\d,.]+)')
df['DEVISENKURS'] = df['DEVISENKURS'].str.replace('.', '').str.replace(',', '.')
df['DEVISENKURS'] = df['DEVISENKURS'].fillna(1)

df['HANDELSTAG'] = df['Text'].str.extract(r'((?<=HANDELSTAG )[\d.]+)')


df['Type'] = df['Text'].str.extract(r'((?<=WERTPAPIERABRECHNUNG )\S+)')
df.loc[df['Text'].str.contains('VERMOEGENSDEPOT EINZAHLUNG'), 'Type'] = 'Einzahlung'
df.loc[df['Text'].str.contains('GNISGUTSCHRIFT INVESTMENTFONDS'), 'Type'] = 'Dividende'
df.loc[df['Text'].str.contains('VERMOEGENSDEPOT SPARPLAN'), 'Type'] = 'Sparplan'


df['WKN'] = df['Text'].str.extract(r'((?<=WKN )\S+)')
df['ISIN'] = df['Text'].str.extract(r'((?<=\/ )\S+)')
df['Name'] = df['Text'].str.extract(r'((?<=\/ ).+?(?= DEPOTNR))')

for row_iter in df.iterrows():
    try:
        df.loc[row_iter[0], 'Name'] = row_iter[1]['Name'].replace(row_iter[1]['ISIN'] + ' ', '')
    except:
        pass

df['Kurs'] = df['Kurs'].fillna(0)
df['Menge'] = df['Menge'].fillna(0)

df['Betrag_'] = df['Betrag'].str.extract(r'([+-])') + df['Betrag'].str.extract(r'([\d,.]+)')
df['Betrag_'] = df['Betrag_'].str.replace(r'.', '').str.replace(',', '.')
df['Betrag_'] = df['Betrag_'].astype('float')
    

df['faktor'] = 1
df.loc[df['Type'].str.contains('VERKAUF') == True, 'faktor'] = -1

df['Kosten'] = df['Menge'].astype('float') * df['Kurs'].astype('float') / df['DEVISENKURS'].astype('float') + df['Betrag_'] * df['faktor']
df['Kosten'] = df['Kosten'].round(2)

df.loc[df['Type'].isin(['KAUF', 'VERKAUF']) == False, 'Kosten'] = 0

df.tail(11)

,Text,Buchung,PN-Nr.,Wert,NaN,Betrag,Menge,Kurs,DEVISENKURS,HANDELSTAG,Type,WKN,ISIN,Name,Betrag_,Kosten,faktor
19,EFFEKTEN MLP Banking AG Lisa und Stefan Krone ...,24.09.2020,925,23.09.2020,EUR,"349,78 -",6.5380,53.50000000,1,21.09.2020,KAUF,A0MUWS,DE000A0MUWS7,ZINSPLUS,-349.78,0.00,1
20,EFFEKTEN MLP Banking AG Lisa und Stefan Krone ...,24.09.2020,925,24.09.2020,EUR,"2.029,99 -",43.6370,46.52000000,1,21.09.2020,KAUF,A0BL36,LU0171307068,BGF-WLD HEALTHSC.NA.A2EO,-2029.99,0.00,1
21,EFFEKTEN MLP Banking AG Lisa und Stefan Krone ...,24.09.2020,925,24.09.2020,EUR,"2.030,00 -",20.9270,113.64000000,1.1715000,21.09.2020,KAUF,A1H6XK,LU0552385295,MORGAN STAN.I-GL.OPP.ADL,-2030.00,-0.00,1
22,EFFEKTEN MLP Banking AG Lisa und Stefan Krone ...,24.09.2020,925,24.09.2020,EUR,"2.030,02 -",61.6090,32.95000000,1,21.09.2020,KAUF,921800,LU0099574567,FID.FDS-GL TECHNOL.A GL.,-2030.02,-0.00,1
23,EFFEKTEN MLP Banking AG Lisa und Stefan Krone ...,24.09.2020,925,24.09.2020,EUR,"2.029,98 -",29.1080,81.70000000,1.1715000,21.09.2020,KAUF,973792,LU0048816135,INVESCO GREATER CHINA E.A,-2029.98,0.00,1
24,"RETOUREN KRONE, STEFAN UND LISA Lisa und Stefa...",23.09.2020,963,23.09.2020,EUR,"14.853,00 -",0,0,1,NaN,Einzahlung,"A0MUWS,",NaN,NaN,-14853.00,0.00,1
25,EFFEKTEN MLP Banking AG Lisa und Stefan Krone ...,23.09.2020,925,23.09.2020,EUR,"2.029,84 -",10.3690,195.76000000,1,21.09.2020,KAUF,A141Q9,LU1279334053,PICTET - ROBOTICS ICAPEO,-2029.84,-0.00,1
26,EFFEKTEN MLP Banking AG Lisa und Stefan Krone ...,23.09.2020,925,23.09.2020,EUR,"2.174,95 -",7.7360,280.92170000,1,21.09.2020,KAUF,A0YEDG,IE00B5BMR087,ISHSVII-CORE S+P500 DLACC,-2174.95,-1.74,1
27,"LASTSCHRIFTEINR. KRONE, STEFAN UND LISA KRONE ...",23.09.2020,2044,23.09.2020,EUR,"14.850,00 +",0,0,1,NaN,Einzahlung,"A0MUWS,",NaN,NaN,14850.00,0.00,1
28,EFFEKTEN MLP Banking AG Lisa und Stefan Krone ...,04.09.2020,925,03.09.2020,EUR,"149,96 -",2.8150,53.27000000,1,01.09.2020,KAUF,A0MUWS,DE000A0MUWS7,ZINSPLUS,-149.96,-0.00,1


In [162]:
df.loc[df['Type'].str.contains('VERKAUF') == True]

,Text,Buchung,PN-Nr.,Wert,NaN,Betrag,Menge,Kurs,DEVISENKURS,HANDELSTAG,Type,WKN,ISIN,Name,Betrag_,Kosten,faktor
10,EFFEKTENGUTSCHRIFT MLP Banking AG Lisa und Ste...,05.11.2020,925,04.11.2020,EUR,"196,69 +",3.6750,53.52000000,1,02.11.2020,VERKAUF,A0MUWS,DE000A0MUWS7,ZINSPLUS,196.69,-0.0,-1
